## Aggregation function implementation

In our optimization problem we are interested in the problem of minimizing the induce drag coefficient $C_{D_i}$, the structural mass or something else subject to constraints on structural failure. In particular, we are interested in failure criteria based on yield stress. That choise is desirable because checking the failure criteria in the optimization problem will mean that the optimized design of the structure is suitable for the prescribed loading condition.

Including failure constraint directly in the structural optimization problem provides some issues:

As first the resulting size of the problem, in fact we have to set a constraint for each FEM element, that lead to, for detailed high-fidelity structural models, an optimization problem with many of thousands or millions of failure constraints. These constraint are **costly** to enforce because they can only be checked by completing the structural analysis. 

In our example case with the CRM wing model, with about 25'000 FEM elements, the computational cost is so high that already in the openMDAO setting step the software can crash for memory leak.

It is therefore necessary to use aggregation to reduce the number of constraint. The lost common approach to deal with the large number of constraint is constraint aggregation. Following this approach, the local constraint are lumped togheter into a global constraint. Instead of many local constraint, only a single aggregated constraint is considered, wich drastically decreases the computation cost.

In the optimization problem there is another issue; in fact a natural choise for these constraint aggregates is a maximum-value function. However, the maximum-value functions are not differentiable and cannot be used efficiently in conjunction with gradient-based desogn optimization.

Our choise for this problem have been to use the **_Kreisselmeiser-Steinhauser_** function or the **_p-norm_** function.

Let's see in the detail this aggregation function and how we implemented it in the script.

### Aggregation Function

In the optimization process, each time that we run a FEM analysis, we store in a vector all the stress's components over the displacements of the structural nodes. From the stress's vector we want to obain a continuos function, representative of the maximux of the stress for each iteration. 

Several aggregation functions have been used in literature: Kreisselmeiser-Steinhauser function, P-norm function, P-mean function. These aggregation functions have in common that they transform a set of local function values into a scalar function. This scalr function depend on an aggregation parameter $P > 0$, and converges in the limit to the maximum local function value:

\begin{equation} \lim_{P\to\infty} \Psi( f;P ) = max(f_1,f_2,...,f_N)\end{equation}

Here , $f = (f_1,f_2,...,f_N)^T$ denotes a vector in which the entries are the local function values, and $\Psi$ is the scal aggregation function.

Some aggregation function approsimate the maximum local function value from above, and other from below; depending on this charatheristic behavior the aggregation function forms an upper or lower -bound to the maximum local function value.

In the next figure you can see the difference between an upper and a lower bound aggregation and the effect of the draw-down factor $P$:



![title](img/peffect.png)

In this figure there are described the relative error of the scalar function related to the maxim value of the stress. You can also see how some aggregation function converge from above while the other from below. 

**Note**: This figure can be obtained from a script joined in the end, it will be useful for decide the aggregation function, the draw-down factor and the characteristic stress used for the adimensionalization in relation of the dimension of your problem.

Let's see in the particular the aggregation function implemented in the script and the code:

### Preamble of the script

Here there is the defionition of all the variables used in the script.

To work we have to give to the method 3 values:
* the lenght of the stress's vector, usually defined at the start of the script as you can see in the optimization example;
* the draw-down factor, you should decide it in function of the dimension of your problem and the accuracy that u want to reach ( an higher draw-down factor entail an higher computational cost but an higher accuracy);
* the function that u want to use;

The other variables are:
* the Von Mises Stresses Vector **VMStress**;
* the charateristic parameter **s0** used for the adimensionalization of the stress; the adimensionalization need because in relation of the unit of measure that you choose it's possibile to incur in overflow or underflow errors;
* the Aggregation function value **G**.

In [ ]:
    def __init__(self,n_stress,p,function):
        super(Aggregation,self).__init__()
        
        #Number of Von Mises Stresses obtained
        self.n_stress=n_stress
        
        #Draw-down factor
        self.p=p
        
        #Aggregation function type
        self.function=function
        
        #Von Mises stresses vector
        self.add_param('VMStress',np.zeros(self.n_stress))
                
        #Reference vqlue for normalization of stresses
        self.add_param('s0',val=0.0)
        
        #Aggregation function
        self.add_output('G',val=0.0)

### P-norm and P-mean

\begin{equation} \Psi_{PN}^U = \bigg(\sum_{i=1}^{N} f_i^P\bigg)^{1/P} \end{equation}

and

\begin{equation} \Psi_{PM}^L = \bigg(\frac{1}{N}\sum_{i=1}^{N} f_i^P\bigg)^{1/P} \end{equation}

We superscript U and L to denote an upper and lower bounded function.
P-norm and P-mean are used to aggregate non-negative stress criteria, because in the assumpition the local function values need to be non-negative.

The difference between these two aggregation functions is that the P-norm is an upper bound, and the P-mean is a lower bound to the maximum local function value:

\begin{equation} \Psi_{PM}^L \le max (f_1,f_2,...,f_N) \le\Psi_{PN}^U \end{equation}

In [ ]:
        elif function=='Gpn':
            
            for k in range(n_stress):
                
                summ+=(VMStress[k]/gmax)**p
                
            G=gmax*(summ)**(1/p)   
            
                
        elif function=='Gpm':
            
            for k in range(n_stress):
                
                summ+=(VMStress[k]/gmax)**p
                
            G=(summ/n_stress)**(1/p)*gmax  

### Kreisselmeiser-Steinhauser function

\begin{equation} \Psi_{KS}^U = \frac{1}{P}ln\bigg(\sum_{i=1}^{N} e^{P f_i}\bigg) \end{equation}

Here we used the superscript U to emphasize thzt the KS-function forms an upper bound to the maximum local function value. For $P>0$ the function overstimates the maximum local function value.

\begin{equation} \Psi_{KS}^L = \frac{1}{P}ln\bigg(\frac{1}{N}\sum_{i=1}^{N} e^{P f_i}\bigg) \end{equation}

Subtracting the maximum diufference $\frac{1}{P} ln(N)$ giving a lower bound to the maximum local function value, obtaining the KS-function lower bounded denoted with the superscript L.

In the code we implemented a unified version of this aggregation function to avoid numerical error caused by the logarithm:

\begin{equation} \Psi_{KS}^U = f_{max} +\frac{1}{P}ln\bigg(\sum_{i=1}^{N} e^{ (f_i-f_{max})}\bigg) \end{equation}



<center>and</center>


\begin{equation} \Psi_{KS}^L = f_{max} +\frac{1}{P}ln\bigg(\sum_{i=1}^{N} e^{ (f_i-f_{max})}\bigg)- \frac{\ln N}{P} \end{equation} 

In [ ]:
        if function=='Gksl':
        
            for k in range (n_stress):
                
                summ+=np.exp(p*((VMStress[k]-gmax)/s0))
                
            G=((gmax/s0)+(np.log(summ)/p)-(np.log(n_stress)/p))*s0
                    
        
        elif function=='Gksu':
            
            for k in range (n_stress):
                
                summ+=np.exp(p*((VMStress[k]-gmax)/s0))
                
                G=((gmax/s0)+(np.log(summ)/p))*s0

### Output setting

In the componet we set as output the aggregation function G and we save it in a dictonary to collect the data.

This output G will be used in the definition of the constraint function to replace the single components of the Von Mises Stresses Vector.

In [ ]:
        #Set the aggregation function as an output
        unknowns['G']=G
        
        output_aggr = {}

        output_aggr['G'] = G


        return output_aggr

### How to choose the draw-down factor P and the function

There is a test code that we used during the implementation. That code can be use to test the aggregation function and the effect of P using your Stress vector. 

#### Importing section

In [1]:

                
# -*- coding: utf-8 -*-
"""
Created on Tue May 29 09:55:22 2018

@author: a.iacono
"""

from __future__ import print_function

import numpy as np

from openmdao.api import Component,Group,Problem,IndepVarComp,view_tree

import matplotlib.pyplot as plt

#### Aggregation Compoennt in openMDAO language

This is the component defined as in the optimization script:

In [ ]:
#Component which gives the aggregation function (G) given the Von Mises Stresses
class Aggregation(Component):
    def __init__(self,n_stress,p,function):
        super(Aggregation,self).__init__()
        #Number of Von Mises Stresses obtained
        self.n_stress=n_stress
        #Draw-down factor
        self.p=p
        #Aggregation function type
        self.function=function
        #Von Mises stresses vector
        self.add_param('VMStress',np.zeros(self.n_stress))
        #Reference vqlue for normalization of stresses
        self.add_param('s0',val=0.0)
        #Aggregation function
        self.add_output('G',val=0.0)
        
    def solve_nonlinear(self,params,unknowns,resids):
        n_stress=self.n_stress
        p=self.p
        function=self.function
        VMStress=params['VMStress']
        s0=params['s0']
        gmax=max(VMStress)
        summ=0.0
        if function=='Gksl':
            for k in range (n_stress):
                summ+=np.exp(p*((VMStress[k]-gmax)/s0))
            G=((gmax/s0)+(np.log(summ)/p)-(np.log(n_stress)/p))*s0
        elif function=='Gksu':
            for k in range (n_stress):
                summ+=np.exp(p*((VMStress[k]-gmax)/s0))
                G=((gmax/s0)+(np.log(summ)/p))*s0
        elif function=='Gpn':
            for k in range(n_stress):
                summ+=(VMStress[k]/gmax)**p
            G=gmax*(summ)**(1/p)   
        elif function=='Gpm':
            for k in range(n_stress):
                summ+=(VMStress[k]/gmax)**p
            G=(summ/n_stress)**(1/p)*gmax  
        #Set the aggregation function as an output
        unknowns['G']=G
        output_aggr = {}
        output_aggr['G'] = G

Now we have to define the stress vector. Normally during the process is automatically obtained from the source, but we want just to run the aggregation using a sample vector, so we create a vector named **VMStress** from a local variable. In the folder there is a spydata, this file contains some sample stress vectors named **VMSi** $i=(1,2,...,10)$, but u can use your own stress vector saved as global variables in your python editor.

NOTE: If you don't load any variables the script cannot run

In [ ]:
VMStress=np.asarray(VMS1)

#### Problem definition

Here we are setting the initial value for the draw-down factor P, the value of the caratheristic stress ( set it in relation of the unity that u are using for your case) and we inizialize a storage vector for the plots.

In [ ]:
p=1.0
s0=40000000.0
gvec=np.array([])

#### Running the code

Now we are running the component to obation the aggregation function value for different values of p; set the while limit as you want:

In [ ]:
while p<=200:
    p+=1

    if __name__ == "__main__":
    
        n_stress=len(VMStress)
        function='Gksl'
        #('Gksl','Gksu','Gpn','Gpm')
        
        
        top = Problem()
        top.root = root = Group()
        root.add('vonmises', IndepVarComp('VMStress', VMStress), promotes=['*'])
        root.add('reference_s', IndepVarComp('s0', s0), promotes=['*'])
        root.add('aggregation',Aggregation(n_stress,p,function),promotes=['*'])
        
        
        top.setup()
        view_tree(top, show_browser=False)
        top.run()
        Error=(top['G']-max(VMStress))/max(VMStress)*100
        gvec=np.append(gvec,Error)
            
        
    
        print ("%s =  %f" % (function, top['G']))
        
    
        Max_VM=max(VMStress)
        G=top['G']

#### Plotting section

Here we create and save in the folder the plots about the iterations:

In [ ]:
    
pp = np.arange(1., p, 1)
plt.plot(pp,gvec)
plt.grid(True)
plt.text(p/2,(gvec[1]+gvec[-2])/2,function,fontsize=18)
plt.xlabel('p', fontsize=14)
plt.ylabel('Error [%]', fontsize=14)
#plt.xlim(pp.min() * 1.5, pp.max() * 1.1)
#plt.ylim(95, 110)
plt.show
plt.savefig('%s_p=%d.png'%(function,p-1),dpi=1000, bbox_inches='tight')
